In [1]:
import json
import requests
import os
import re
from typing import List, Dict, Any

In [2]:
from together import Together

In [22]:
def generate_params_dict(
    prompt: str, 
    temperature: float = 1, 
    role = 'user',
    top_p: float = 1,
    max_tokens: int = 500,
    model: str = "meta-llama/Llama-3.2-3B-Instruct-Turbo"
):
    """
    Call an LLM with different sampling parameters to observe their effects.
    
    Args:
        prompt: The text prompt to send to the model
        temperature: Controls randomness (lower = more deterministic)
        top_p: Controls diversity via nucleus sampling
        max_tokens: Maximum number of tokens to generate
        model: The model to use
        
    Returns:
        The LLM response
    """
    
    # Create the dictionary with the necessary parameters
    kwargs = {"prompt": prompt, 'role':role, "temperature": temperature, "top_p": top_p, "max_tokens": max_tokens, 'model': model} 


    return kwargs

In [20]:
def generate_with_single_input(prompt: str, 
                               role: str = 'user', 
                               top_p: float = 1, 
                               temperature: float = 1,
                               max_tokens: int = 500,
                               model: str ="meta-llama/Llama-3.2-3B-Instruct-Turbo",
                               together_api_key = None,
                              **kwargs):
    
    if top_p is None:
        top_p = 1
    if temperature is None:
        temperature = 1

    payload = {
            "model": model,
            "messages": [{'role': role, 'content': prompt}],
            "top_p": top_p,
            "temperature": temperature,
            "max_tokens": max_tokens,
            **kwargs
                  }
    if (not together_api_key) and ('TOGETHER_API_KEY' not in os.environ):
        url = os.path.join('https://proxy.dlai.link/coursera_proxy/together', 'v1/chat/completions')   
        response = requests.post(url, json = payload, verify=False)
        if not response.ok:
            raise Exception(f"Error while calling LLM: f{response.text}")
        try:
            json_dict = json.loads(response.text)
        except Exception as e:
            raise Exception(f"Failed to get correct output from LLM call.\nException: {e}\nResponse: {response.text}")
    else:
        if together_api_key is None:
            together_api_key = os.environ['TOGETHER_API_KEY']
        client = Together(api_key =  together_api_key)
        json_dict = client.chat.completions.create(**payload).model_dump()
        json_dict['choices'][-1]['message']['role'] = json_dict['choices'][-1]['message']['role'].name.lower()
    try:
        output_dict = {'role': json_dict['choices'][-1]['message']['role'], 'content': json_dict['choices'][-1]['message']['content']}
    except Exception as e:
        raise Exception(f"Failed to get correct output dict. Please try again. Error: {e}")
    return output_dict

In [5]:
def generate_with_multiple_input(messages: List[Dict], 
                               top_p: float = 1, 
                               temperature: float = 1,
                               max_tokens: int = 500,
                               model: str ="meta-llama/Llama-3.2-3B-Instruct-Turbo", 
                                together_api_key = None,
                                **kwargs):
    payload = {
        "model": model,
        "messages": messages,
        "top_p": top_p,
        "temperature": temperature,
        "max_tokens": max_tokens,
        **kwargs
              }
    if (not together_api_key) and ('TOGETHER_API_KEY' not in os.environ):
        url = os.path.join('https://proxy.dlai.link/coursera_proxy/together', 'v1/chat/completions')   
        response = requests.post(url, json = payload, verify=False)
        if not response.ok:
            raise Exception(f"Error while calling LLM: f{response.text}")
        try:
            json_dict = json.loads(response.text)
        except Exception as e:
            raise Exception(f"Failed to get correct output from LLM call.\nException: {e}\nResponse: {response.text}")
    else:
        if together_api_key is None:
            together_api_key = os.environ['TOGETHER_API_KEY']
        client = Together(api_key =  together_api_key)
        json_dict = client.chat.completions.create(**payload).model_dump()
        json_dict['choices'][-1]['message']['role'] = json_dict['choices'][-1]['message']['role'].name.lower()
    try:
        output_dict = {'role': json_dict['choices'][-1]['message']['role'], 'content': json_dict['choices'][-1]['message']['content']}
    except Exception as e:
        raise Exception(f"Failed to get correct output dict. Please try again. Error: {e}")
    return output_dict

In [6]:
def call_llm_with_context(prompt: str, context: list,  role: str = 'user', **kwargs):
    """
    Calls a language model with the given prompt and context to generate a response.

    Parameters:
    - prompt (str): The input text prompt provided by the user.
    - role (str): The role of the participant in the conversation, e.g., "user" or "assistant".
    - context (list): A list representing the conversation history, to which the new input is added.
    - **kwargs: Additional keyword arguments for configuring the language model call (e.g., top_k, temperature).

    Returns:
    - response (str): The generated response from the language model based on the provided prompt and context.
    """

    # Append the dictionary {'role': role, 'content': prompt} into the context list
    context.append({'role': role, 'content': prompt})

    # Call the llm with multiple input passing the context list and the **kwargs
    response = generate_with_multiple_input(context, **kwargs)

    # Append the LLM response in the context dict
    context.append(response) 
    
    return response

In [7]:
os.environ['TOGETHER_API_KEY'] = "76d81fd6490bf6aa4277b46347bf1af6b951e468bedd5eef3abc37b851d75b31"

In [8]:
def check_if_outfit_or_supplement(query):
    prompt = f"""
Determine the category of the following query as either "nutritional" or "outfit" related.
- Nutritional queries: These are related to nutrition products, such as whey protein, vitamins, supplements, dietary products, and health-related food and beverages.
  - Outfit queries: These pertain to clothing and fashion, including items like shirts, dresses, shoes, accessories, and jewelry.
Examples:

1. Query: “Where can I buy high-protein snacks?” Expected answer: Nutritional
2. Query: “Best shirt styles for summer 2023” Expected answer: Outfit
3. Query: “Are there any shoes designed for running?” Expected answer: Outfit
4. Query: “What multivitamins should I take daily?” Expected answer: Nutritional
5. Query: “Best weight loss products that are stylish” Expected answer: Nutritional
6. Query: “Athletic wear that boosts performance” Expected answer: Outfit 

Query: {query}

Instructions: Respond with “Nutritional” if the query pertains to nutritional products or “Outfit” if it pertains to clothing or fashion products.
Answer only one single word.
"""
    return prompt

In [11]:
query = "Give me the available vitamins supplement you have in your catalogue."
test = check_if_outfit_or_supplement(query)
print(test)


Determine the category of the following query as either "nutritional" or "outfit" related.
- Nutritional queries: These are related to nutrition products, such as whey protein, vitamins, supplements, dietary products, and health-related food and beverages.
  - Outfit queries: These pertain to clothing and fashion, including items like shirts, dresses, shoes, accessories, and jewelry.
Examples:

1. Query: “Where can I buy high-protein snacks?” Expected answer: Nutritional
2. Query: “Best shirt styles for summer 2023” Expected answer: Outfit
3. Query: “Are there any shoes designed for running?” Expected answer: Outfit
4. Query: “What multivitamins should I take daily?” Expected answer: Nutritional
5. Query: “Best weight loss products that are stylish” Expected answer: Nutritional
6. Query: “Athletic wear that boosts performance” Expected answer: Outfit 

Query: Give me the available vitamins supplement you have in your catalogue.

Instructions: Respond with “Nutritional” if the query pe

In [21]:
# Testing a simple query
query = "Give me the available vitamins supplement you have in your catalogue."
generate_with_single_input(check_if_outfit_or_supplement(query), max_tokens = 2)

{'role': 'assistant', 'content': 'Nutritional'}

In [23]:
# ASCII color codes
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

queries = [
    {"query": "Where can I buy whey protein?", "label": "Nutritional"},
    {"query": "Recommended vitamins for winter", "label": "Nutritional"},
    {"query": "Latest fashion for women's dresses", "label": "Outfit"},
    {"query": "Comfortable sneakers for daily use", "label": "Outfit"},
    {"query": "Best energy bars for athletes", "label": "Nutritional"},
    {"query": "Trendy accessories for men", "label": "Outfit"},
    {"query": "Low-carb diet food options", "label": "Nutritional"},
    {"query": "What supplements help with muscle recovery?", "label": "Nutritional"},
    {"query": "Casual wear that supports healthy living", "label": "Outfit"}
]

for item in queries:
    query = item["query"]
    prompt = check_if_outfit_or_supplement(query)
    expected_label = item["label"]
    response = generate_with_single_input(prompt, max_tokens = 2)
    result = response['content']
    
    # Determine color based on comparison
    if result == expected_label:
        color = GREEN
    else:
        color = RED

    print(f"Query: {query}\nResult: {result}\nExpected: {color}{expected_label}{RESET}\n")
    

Query: Where can I buy whey protein?
Result: Nutritional
Expected: Nutritional

Query: Recommended vitamins for winter
Result: Nutritional
Expected: Nutritional

Query: Latest fashion for women's dresses
Result: Outfit
Expected: Outfit

Query: Comfortable sneakers for daily use
Result: Outfit
Expected: Outfit

Query: Best energy bars for athletes
Result: Nutritional
Expected: Nutritional

Query: Trendy accessories for men
Result: Outfit
Expected: Outfit

Query: Low-carb diet food options
Result: Nutritional
Expected: Nutritional

Query: What supplements help with muscle recovery?
Result: Nutritional
Expected: Nutritional

Query: Casual wear that supports healthy living
Result: Outfit
Expected: Outfit



In [24]:
def decide_if_technical_or_creative(query):
    """
    Determines whether a given query is creative or technical in nature.

    Args:
        query (str): The query string to be evaluated.

    Returns:
        str: A label indicating the query type, either 'creative' or 'technical'.

    This function constructs a prompt to classify a query based on its content. 
    Creative queries typically involve requests to generate original content, whereas 
    technical queries relate to documentation or technical information, such as procedures.
    By leveraging an LLM, it identifies the query type and returns an appropriate label.
    """
    
    PROMPT = f"""Decide if the following query is a creative query or a technical query.
    Creative queries ask you to create content, while technical queries are related to documentation or technical requests, like information about procedures.
    Answer only 'creative' or 'technical'.
    Query: {query}
    """
    result = generate_with_single_input(PROMPT)
    label = result['content']
    return label

In [25]:
queries = ["What is Pi-hole?", 
           "Suggest to me three places to visit in South America"]
for query in queries:
    label =decide_if_technical_or_creative(query)
    print(f"Query: {query}, label: {label}")

Query: What is Pi-hole?, label: technical
Query: Suggest to me three places to visit in South America, label: Creative


In [26]:
def answer_query(query):
    """
    Processes a query and generates an appropriate response by categorizing the query
    as either 'technical' or 'creative', and modifies behavior based on this categorization.

    Args:
        query (str): The query string to be answered.

    Returns:
        str: A generated response from the LLM tailored to the nature of the query.

    This function first determines the nature of the query using the `decide_if_technical_or_creative` function. 
    If the query is classified as 'technical', it sets parameters suitable for precise and low-variability responses. 
    If the query is 'creative', it applies parameters allowing for more variability and creativity. 
    If the classification is inconclusive, it uses neutral parameters. 
    It then generates a response using these parameters and returns the content.
    """
    
    # Determine whether the query is 'technical' or 'creative'
    label = decide_if_technical_or_creative(query).lower()

    # Set parameters for technical queries (precise, low randomness)
    if label == 'technical':
        kwargs = generate_params_dict(query, temperature=0, top_p=0.1)
    
    # Set parameters for creative queries (variable, high randomness)
    elif label == 'creative':
        kwargs = generate_params_dict(query, temperature=1.1, top_p=0.4)

    # Use default parameters if the query type is inconclusive
    else:
        kwargs = generate_params_dict(query, temperature=0.5, top_p=0.5)
    
    # Generate a response based on the query type and parameters
    response = generate_with_single_input(**kwargs)
    
    # Extract and return the content from the response
    result = response['content']
    return result

In [27]:
queries = ["What is Pi-hole?", 
           "Suggest to me three places to visit in South America"]
for query in queries:
    result = answer_query(query)
    print(f"Query: {query}\nAnswer: {result}\n\n#######\n")

Query: What is Pi-hole?
Answer: Pi-hole is a free, open-source, and self-contained ad-blocking DNS server that can be used to block advertisements on any network. It's a simple, yet powerful solution for blocking ads, tracking scripts, and other online threats.

Here's how it works:

1. Pi-hole acts as a DNS (Domain Name System) server, which translates domain names into IP addresses that computers can understand.
2. When a device on the network requests a website, Pi-hole intercepts the request and checks the domain name against its database of blocked ads and trackers.
3. If the domain name is found in the blocklist, Pi-hole returns a fake IP address that directs the user to a non-existent website, effectively blocking the ad.
4. Pi-hole can be configured to block ads on specific websites, domains, or even entire networks.

Pi-hole is often used in home networks, small businesses, and organizations to:

* Block ads and improve browsing speed
* Reduce online tracking and data collecti

In [28]:
def generate_system_call(command):
    PROMPT = f"""
You are an assistant program that converts natural language commands into structured JSON for controlling smart home devices. The JSON should conform to a specific format describing the device, action, and parameters. Here's how you can do it:

**Available Devices and Actions:**

1. **Light**
   - Actions: "turn on", "turn off"
   - Parameters: color, intensity (percentage)

2. **Automatic Lock**
   - Actions: "lock", "unlock"
   - Parameters: None

3. **Sound System (Speaker)**
   - Actions: "play", "pause", "stop", "set volume"
   - Parameters: volume (integer), track (string), playlist_style (string)

4. **TV**
   - Actions: "turn on", "turn off", "change channel", "adjust volume"
   - Parameters: channel (string), volume (integer)

5. **Air Conditioner**
   - Actions: "turn on", "turn off", "set temperature", "adjust fan speed"
   - Parameters: temperature (integer), fan_speed (low/medium/high)

**Rooms and Devices:**
- **Office**
  - Lights: "office_light_1" (ID: 123), "office_light_2" (ID: 321)
  - Automatic Lock: "office_door_lock" (ID: 111)

- **Living Room**
  - Light: "living_room_light" (ID: 222)
  - Speaker: "living_room_speaker" (ID: 223)
  - Air Conditioner: "living_room_airconditioner" (ID: 556)

- **Kitchen**
  - Light: "kitchen_light" (ID: 333)

- **Bedroom**
  - Light: "bedroom_light" (ID: 444)
  - TV: "bedroom_tv" (ID: 445)

- **Bathroom**
  - Light: "bathroom_light" (ID: 555)

**Task:**
Convert the following natural language command into the structured JSON format based on the available devices:

**Input Examples:**

1. "Turn on the office light with ID 123 with blue color and 50% intensity."
   - JSON:
     [
     {{
       "room": "office",
       "object_id": "123",
       "object_name": "office_light_1",
       "action": "turn on",
       "parameters": {{"color": "blue", "intensity": "50%"}}
     }}
     ]

2. "Lock the office door."
   - JSON:
   [
     {{
       "room": "office",
       "object_id": "111",
       "object_name": "office_door_lock",
       "action": "lock",
       "parameters": {{}}
     }}
    ]

2. "Make my living room a cheerful place"
   - JSON:
   [
     {{
       "room": "living_room",
       "object_id": "222",
       "object_name": "living_room_light",
       "action": "turn on",
       "parameters": {{'intensity': '80%', 'color':'yellow'}}
     }},
     {{
       "room": "living_room",
       "object_id": "223",
       "object_name": "living_room_speaker",
       "action": "turn on",
       "parameters": {{'volume': '100', 'playlist_style':'party'}}
     }},
     
   ]

**Note:**
- Ensure that each JSON object correctly maps the natural command to the appropriate device and action using the listed device ID.
- Use the object ID to differentiate between devices when the room contains multiple similar items.
- You can add more than one parameter in the parameters dictionary.

Using this information, translate the following command into JSON: "{command}". Output a list with all the necessary JSONs. 
Always output a list even if there is only one command to be applied, do not output anything else but the desired structure.
"""
    kwargs = generate_params_dict(PROMPT, temperature=0.4, top_p=0.1)
    result = generate_with_single_input(**kwargs)
    return result['content']

In [29]:
print(generate_system_call("Play a chill playlist very loud"))

[
  {
    "room": "living_room",
    "object_id": "223",
    "object_name": "living_room_speaker",
    "action": "turn on",
    "parameters": {"volume": "100", "playlist_style": "chill"}
  },
  {
    "room": "living_room",
    "object_id": "223",
    "object_name": "living_room_speaker",
    "action": "set volume",
    "parameters": {"volume": "100"}
  }
]


In [30]:
print(generate_system_call("I'm tired today, please make my living room a very cozy ambient, it is really cold today too."))

[
  {
    "room": "living_room",
    "object_id": "222",
    "object_name": "living_room_light",
    "action": "turn on",
    "parameters": {"intensity": "100%", "color": "warm white"}
  },
  {
    "room": "living_room",
    "object_id": "223",
    "object_name": "living_room_speaker",
    "action": "turn on",
    "parameters": {"volume": "100", "playlist_style": "relaxing"}
  },
  {
    "room": "living_room",
    "object_id": "556",
    "object_name": "living_room_airconditioner",
    "action": "turn on",
    "parameters": {"temperature": "22", "fan_speed": "low"}
  }
]


In [31]:
from pydantic import BaseModel, validator, conint, Field
from typing import Literal, Union, Optional, List
import json

# Define the schema for the output
class VoiceNote(BaseModel):
    title: str = Field(description="A title for the voice note")
    summary: str = Field(description="A short one sentence summary of the voice note.")
    actionItems: list[str] = Field(
        description="A list of action items from the voice note"
    )


In [32]:
transcript = (
        "Good morning! It's 7:00 AM, and I'm just waking up. Today is going to be a busy day, "
        "so let's get started. First, I need to make a quick breakfast. I think I'll have some "
        "scrambled eggs and toast with a cup of coffee. While I'm cooking, I'll also check my "
        "emails to see if there's anything urgent."
    )


messages=[
            {
                "role": "system",
                "content": "The following is a voice message transcript. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": transcript,
            },
        ]

response_format={
            "type": "json_schema",
            "schema": VoiceNote.model_json_schema(),
        }

result = generate_with_multiple_input(messages, response_format = response_format)
result_json = json.loads(result['content'])
print(json.dumps(result_json, indent=2))

{
  "title": "Morning Routine",
  "summary": "Preparing breakfast at 7:00 AM, checking emails.",
  "actionItems": [
    "> Eat scrambled eggs and toast",
    "> Drink a cup of coffee",
    "> Check emails"
  ]
}
